In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def clean_file(code):
  #no multiline comments
  clean_code=re.sub(r'/\*(.*?)\*/', '', code, flags=re.DOTALL)
  #no tabs
  clean_code=re.sub(r'\t',' ',clean_code)
  #no empty lines
  clean_code = re.sub(r'\n(\s*)\n', '\n', clean_code)
  clean_code= re.sub(r'^\n+','',clean_code)
  #lines of code
  matches=re.findall('\n',clean_code)
  return clean_code,len(matches)

In [ ]:
file=open("/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-01/original/T1.java")
og,og_linesOfCode=clean_file(file.read())
print(og)
print(og_linesOfCode)

public class T1 {
 public static void main(String[] args) {
  System.out.println("Welcome to Java");
  System.out.println("Welcome to Java");
  System.out.println("Welcome to Java");
  System.out.println("Welcome to Java");
  System.out.println("Welcome to Java");
 }
}

9


In [ ]:
file=open("/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-01/plagiarized/L1/03/Main.java")
plag,plag_linesOfCode=clean_file(file.read())
print(plag)
print(plag_linesOfCode)

public class Main {
    public static void main(String[] args) { 
            System.out.println("Welcome to Java"); 
            System.out.println("Welcome to Java"); 
            System.out.println("Welcome to Java"); 
            System.out.println("Welcome to Java"); 
            System.out.println("Welcome to Java");
    }
}

9


In [ ]:
file=open("/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-01/non-plagiarized/03/Nomor1_2451041557A22145B3701B0184109CAB013.java",'r')
nonPlag,nonPlag_linesOfCode=clean_file(file.read())
print(nonPlag)
print(nonPlag_linesOfCode)

public class Nomor1_2451041557A22145B3701B0184109CAB013 {
    public static void main(String[] args) {
        for(int i=0;i<5;i++)
        {
            System .out.println("Welcome To Java");
        }
    }
}

8


In [ ]:
# def count_lines_of_code(file_path):
#     line_count = 0

#     with open(file_path, 'r') as file:
#         lines = file.readlines()
#         print(len(lines))

#     for i in range(0,len(lines)):
#         line = lines[i].strip()

#         # Skip empty lines and single-line comments
#         if line == '' or line.startswith('//'):
#             continue

#         # Skip multi-line comments
#         if line.startswith('/*'):
#             for j in range(i,len(lines)):
#                 line2 = lines[j].strip()
#                 line_count += -1
#                 if(line2.endswith('*/')):
#                   break  

#         line_count += 1

#     return line_count


In [ ]:
# java_file_path = '/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-01/non-plagiarized/03/Nomor1_2451041557A22145B3701B0184109CAB013.java'
# nonplag_lines_of_code = count_lines_of_code(java_file_path)
# print(nonplag_lines_of_code)

19
8


In [ ]:
import re

def count_loops(code):
    # Regular expression pattern to match loop structures
    loop_pattern = r'(?:for|while|do\s*while)'

    # Count the number of loop occurrences in the Java code
    loop_count = len(re.findall(loop_pattern, code))

    return loop_count

In [ ]:
print(count_loops(og))
print(count_loops(plag))
print(count_loops(nonPlag))

0
0
1


In [ ]:
import pandas as pd
data=[]
data.append(re.sub(r'\n',' ',og))
# data.append(re.sub(r'\n',' ',plag))
data.append(re.sub(r'\n',' ',nonPlag))
df=pd.DataFrame(data, columns=['code'])


In [ ]:
print(data)

['public class T1 {  public static void main(String[] args) {   System.out.println("Welcome to Java");   System.out.println("Welcome to Java");   System.out.println("Welcome to Java");   System.out.println("Welcome to Java");   System.out.println("Welcome to Java");  } } ', 'public class Nomor1_2451041557A22145B3701B0184109CAB013 {     public static void main(String[] args) {         for(int i=0;i<5;i++)         {             System .out.println("Welcome To Java");         }     } } ']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(data)

In [ ]:
tfidf_tokens=vectorizer.get_feature_names_out()

In [ ]:
df_tfidfvect = pd.DataFrame(data = X.toarray(),index = ['Doc1','Doc2'],columns = tfidf_tokens)

In [ ]:
df_tfidfvect

,args,class,for,int,java,main,nomor1_2451041557a22145b3701b0184109cab013,out,println,public,static,string,system,t1,to,void,welcome
Doc1,0.078573,0.078573,0.000000,0.000000,0.392867,0.078573,0.000000,0.392867,0.392867,0.157147,0.078573,0.078573,0.392867,0.110432,0.392867,0.078573,0.392867
Doc2,0.213560,0.213560,0.300151,0.300151,0.213560,0.213560,0.300151,0.213560,0.213560,0.427120,0.213560,0.213560,0.213560,0.000000,0.213560,0.213560,0.213560


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(X[0].toarray(),X[1].toarray() ))

#plagarised

[[0.99084936]]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(X[0].toarray(),X[1].toarray() ))

#nonPlag

[[0.67120608]]


In [ ]:
df_tfidfvect.keys()

Index(['args', 'class', 'for', 'int', 'java', 'main',
       'nomor1_2451041557a22145b3701b0184109cab013', 'out', 'println',
       'public', 'static', 'string', 'system', 't1', 'to', 'void', 'welcome'],
      dtype='object')

In [ ]:
X.toarray()[0]

array([0.0785734 , 0.0785734 , 0.        , 0.        , 0.39286701,
       0.0785734 , 0.        , 0.39286701, 0.39286701, 0.15714681,
       0.0785734 , 0.0785734 , 0.39286701, 0.11043218, 0.39286701,
       0.0785734 , 0.39286701])